In [71]:

import tifffile as tf
import zarr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.spatial import KDTree
import numpy as np
import pandas as pd
import zarr
from scipy.spatial import KDTree
import tifffile as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import dask.array as da
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.cluster import KMeans

from tifffile import imwrite, tiffcomment


from ome_types import from_tiff


In [72]:
meta = from_tiff('/Users/swarchol/Downloads/WD-76845-097_mask_pyr.ome.ome.tiff')

In [73]:
meta.to_xml()

'<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd" UUID="urn:uuid:05e74433-1a41-480d-b4e4-34c2409885cf" Creator="tifffile.py 2024.9.20">\n  <Image ID="Image:0" Name="Image0">\n    <Pixels ID="Pixels:0" DimensionOrder="XYZCT" Type="uint32" SignificantBits="32" Interleaved="false" BigEndian="true" SizeX="26139" SizeY="27120" SizeZ="1" SizeC="1" SizeT="1" PhysicalSizeX="1.0" PhysicalSizeXUnit="m" PhysicalSizeY="1.0" PhysicalSizeYUnit="m">\n      <Channel ID="Channel:0:0" SamplesPerPixel="1">\n        <LightPath/>\n      </Channel>\n      <TiffData IFD="0" PlaneCount="1">\n        <UUID FileName="WD-76845-097_mask_pyr.ome.ome.tiff">urn:uuid:05e74433-1a41-480d-b4e4-34c2409885cf</UUID>\n      </TiffData>\n    </Pixels>\n  </Image>\n  <StructuredAnnotations>\n    <MapAnnotation ID="Annotation:Resolut

In [55]:
new_xml = """
<OME
	xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
	xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd" UUID="urn:uuid:a4b434fd-eabd-43bb-8164-bcc96c26c2d4" Creator="tifffile.py 2024.9.20">\n  
	<Image ID="Image:0" Name="Image0">\n    
		<Pixels ID="Pixels:0" DimensionOrder="XYZCT" Type="uint32" SignificantBits="32" Interleaved="false" BigEndian="true" SizeX="26139" SizeY="27120" SizeZ="1" SizeC="1" SizeT="1" PhysicalSizeX="1.0" PhysicalSizeXUnit="µm" PhysicalSizeY="1.0" PhysicalSizeYUnit="µm">\n      
			<Channel ID="Channel:0:0" SamplesPerPixel="1">\n        
				<LightPath/>\n      
			</Channel>\n      
			<TiffData IFD="0" PlaneCount="1">\n        
				<UUID FileName="tiled-mask.ome.tif">urn:uuid:a4b434fd-eabd-43bb-8164-bcc96c26c2d4</UUID>\n      
			</TiffData>\n    
		</Pixels>\n  
	</Image>\n  
	<StructuredAnnotations>\n    
		<MapAnnotation ID="Annotation:Resolution:0" Namespace="openmicroscopy.org/PyramidResolution">\n      
			<Value>\n        
				<M K="1">13070 13560</M>\n        
				<M K="2">6535 6780</M>\n        
				<M K="3">3268 3390</M>\n        
				<M K="4">1634 1695</M>\n        
				<M K="5">817 848</M>\n      
			</Value>\n    
		</MapAnnotation>\n  
	</StructuredAnnotations>\n
</OME>
"""

In [56]:
tiffcomment('/Users/swarchol/Downloads/tiled-mask.ome.tif', new_xml.encode('ascii', 'ignore'))


In [79]:
til_io = tf.TiffFile('/Volumes/Simon/Greg/segmentation_mask/WD-76845-097.ome.tiff',is_ome=True)
til_zarr = zarr.open(til_io.series[0].aszarr())
for i in range(6):
    print(til_zarr[i].shape, til_zarr[i].dtype)

(27120, 26139) uint32
(13560, 13069) uint32
(6780, 6534) uint32
(3390, 3267) uint32
(1695, 1633) uint32
(847, 816) uint32


In [75]:
new_pyr_xml = """
<OME
	xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
	xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd" UUID="urn:uuid:05e74433-1a41-480d-b4e4-34c2409885cf" Creator="tifffile.py 2024.9.20">\n  
	<Image ID="Image:0" Name="Image0">\n    
		<Pixels ID="Pixels:0" DimensionOrder="XYZCT" Type="uint32" SignificantBits="32" Interleaved="false" BigEndian="true" SizeX="26139" SizeY="27120" SizeZ="1" SizeC="1" SizeT="1" PhysicalSizeX="1.0" PhysicalSizeXUnit="m" PhysicalSizeY="1.0" PhysicalSizeYUnit="m">\n      
			<Channel ID="Channel:0:0" SamplesPerPixel="1">\n        
				<LightPath/>\n      
			</Channel>\n      
			<TiffData IFD="0" PlaneCount="1">\n        
				<UUID FileName="WD-76845-097_mask_pyr.ome.ome.tiff">urn:uuid:05e74433-1a41-480d-b4e4-34c2409885cf</UUID>\n      
			</TiffData>\n    
		</Pixels>\n  
	</Image>\n  
	<StructuredAnnotations>\n    
		<MapAnnotation ID="Annotation:Resolution:0" Namespace="openmicroscopy.org/PyramidResolution">\n      
			<Value>\n        
				<M K="1">13070 13560</M>\n        
				<M K="2">6535 6780</M>\n        
				<M K="3">3268 3390</M>\n        
				<M K="4">1634 1695</M>\n        
				<M K="5">817 848</M>\n      
			</Value>\n    
		</MapAnnotation>\n  
	</StructuredAnnotations>\n
</OME>\n'
"""

In [76]:
tiffcomment('/Users/swarchol/Downloads/WD-76845-097_mask_pyr.ome copy.tif', new_xml.encode('ascii', 'ignore'))
